# 한국어 혐오 발언 탐지

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 필요 라이브러리 설치

In [1]:
!pip install transformers
!pip install datasets
!pip install torchmetrics
!pip install accelerate -U
!pip install scikit-learn

## 데이터셋 로드

- 학습, 검증, 테스트 데이터셋 준비
- 라벨 정보

      class_label:
        names:
          0: origin
          1: physical
          2: politics
          3: profanity
          4: age
          5: gender
          6: race
          7: religion
          8: not_hate_speech

In [1]:
from datasets import load_dataset

train = load_dataset("jeanlee/kmhas_korean_hate_speech", split="train")
validation = load_dataset("jeanlee/kmhas_korean_hate_speech", split="validation")
test = load_dataset("jeanlee/kmhas_korean_hate_speech", split="test")

In [2]:
# 데이터 예제 출력

print(train)
print(validation)
print(test)
print(train['text'][0])
print(train['label'][0])

Dataset({
    features: ['text', 'label'],
    num_rows: 78977
})
Dataset({
    features: ['text', 'label'],
    num_rows: 8776
})
Dataset({
    features: ['text', 'label'],
    num_rows: 21939
})
"자한당틀딱들.. 악플질 고만해라."
[2, 4]


In [3]:
# 사전
print(train['text'][0:10])

# 리스트
print(train[0:10])


['"자한당틀딱들.. 악플질 고만해라."', '정치적으로 편향된 평론한은 분은 별로...', '적당히좀 쳐먹지.그랬냐??? 안그래도 문재인 때문에 나라 엉망진창인데...', '"안서는 아재들 풀발기 ㅋㄲㅋ"', '우와 ㅋ 능력자', '맛녀석 콩트보다 약했음맛녀석 애청자로써 70%실력발휘', '주영훈 솔직히 호감임 잉꼬부부로 소문났잖아', '이게주간아이돌이랑머가달라...', '아오 슈박 회사생활도 졑깥고 돈벌기 힘들어 죽겠구만 뭔 저딴것들 자꾸 tv나와서 사람 짜증나게하냐 외국서 편히살려면 아닥하고 살아라 대한민국서 취미로 돈벌어가지말고 좀 끄지라고!', '"문재인 하는게 뭐 별거있냐?ㅂㅅㅅㅋ가 하는짓인데 어련하겠어.ㅋㅋㅋ"']
{'text': ['"자한당틀딱들.. 악플질 고만해라."', '정치적으로 편향된 평론한은 분은 별로...', '적당히좀 쳐먹지.그랬냐??? 안그래도 문재인 때문에 나라 엉망진창인데...', '"안서는 아재들 풀발기 ㅋㄲㅋ"', '우와 ㅋ 능력자', '맛녀석 콩트보다 약했음맛녀석 애청자로써 70%실력발휘', '주영훈 솔직히 호감임 잉꼬부부로 소문났잖아', '이게주간아이돌이랑머가달라...', '아오 슈박 회사생활도 졑깥고 돈벌기 힘들어 죽겠구만 뭔 저딴것들 자꾸 tv나와서 사람 짜증나게하냐 외국서 편히살려면 아닥하고 살아라 대한민국서 취미로 돈벌어가지말고 좀 끄지라고!', '"문재인 하는게 뭐 별거있냐?ㅂㅅㅅㅋ가 하는짓인데 어련하겠어.ㅋㅋㅋ"'], 'label': [[2, 4], [8], [2], [4], [8], [8], [8], [8], [3], [2, 3]]}


## 모델 및 토크나이저 로드

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

name_cards= ['monologg/koelectra-base-v3-discriminator', 'kobert-base-v1', 'bert-base-multilingual-cased']
name_card = "monologg/koelectra-base-v3-discriminator"
num_labels = 9
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(name_card, do_lower_case=False)
model = AutoModelForSequenceClassification.from_pretrained(name_card, num_labels=num_labels, problem_type="multi_label_classification")
model.to(device)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [5]:
import torch.nn as nn
from transformers import AutoModel

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=1):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return F_loss.mean()

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model = AutoModel.from_pretrained(name_card)

        self.classifier = nn.Linear(self.model.config.hidden_size, num_labels)
        self.n_classifier = nn.Linear(self.model.config.hidden_size, num_labels)

        # self.intent_loss = nn.BCEWithLogitsLoss()
        self.intent_loss = FocalLoss()
        self.n_loss = nn.CrossEntropyLoss()


    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None, n_label=None):
        model_input= {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
        }

        if token_type_ids is not None:
            model_input['token_type_ids'] = token_type_ids

        hidden = self.model(**model_input)

        # some output dont have pooler_output
        if hasattr(hidden, 'pooler_output'):
            cls = hidden.pooler_output
        else:
            cls = hidden.last_hidden_state[:, 0, :]

        hidden = hidden.last_hidden_state[:, 1:, :]

        intent_logit = self.classifier(hidden.mean(dim=1))
        n_logit = self.n_classifier(cls)

        intent_loss = self.intent_loss(intent_logit, labels)
        num_loss = self.n_loss(n_logit, n_label)

        loss = intent_loss + num_loss

        model_output = {
            'loss': loss,
            'intent_logit': intent_logit,
            'n_logit': n_logit,
        }

        return model_output


model = Model()
model.to(device)

Model(
  (model): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [6]:
# 토크나이저 예시

ids = tokenizer.encode(train['text'][0])
tokenized_words = tokenizer.convert_ids_to_tokens(ids)
model_input = tokenizer(train['text'][0])

print(tokenized_words)
print(ids)
print(model_input)

['[CLS]', '"', '자', '##한', '##당', '##틀', '##딱', '##들', '.', '.', '악', '##플', '##질', '고만', '##해', '##라', '.', '"', '[SEP]']
[2, 6, 3254, 4283, 4403, 4882, 5136, 4006, 18, 18, 3080, 4711, 4152, 25141, 4151, 4118, 18, 6, 3]
{'input_ids': [2, 6, 3254, 4283, 4403, 4882, 5136, 4006, 18, 18, 3080, 4711, 4152, 25141, 4151, 4118, 18, 6, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [7]:
# 토크나이저 예시2

model_input = tokenizer(train['text'][0], max_length=4, truncation=True, padding="max_length")
print(model_input)
reverse = tokenizer.convert_ids_to_tokens(model_input['input_ids'])
print(reverse)

{'input_ids': [2, 6, 3254, 3], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}
['[CLS]', '"', '자', '[SEP]']


In [8]:
# 토크나이저 예시3

print(model_input)

another_name_card = 'roberta-base'
another_tokenizer = AutoTokenizer.from_pretrained(another_name_card, do_lower_case=False)
model_input = another_tokenizer(train['text'][0], max_length=4, truncation=True, padding="max_length")
print(model_input)

another_name_card = 'distilbert-base-uncased'
another_tokenizer = AutoTokenizer.from_pretrained(another_name_card, do_lower_case=False)
model_input = another_tokenizer(train['text'][0], max_length=4, truncation=True, padding="max_length")
print(model_input)


{'input_ids': [2, 6, 3254, 3], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}


{'input_ids': [0, 113, 43998, 2], 'attention_mask': [1, 1, 1, 1]}
{'input_ids': [101, 1000, 100, 102], 'attention_mask': [1, 1, 1, 1]}


In [9]:
from sklearn.preprocessing import MultiLabelBinarizer

def tokenize_function(examples):
    model_input = tokenizer(examples['text'], max_length=128, truncation=True, padding="max_length")

    mlb = MultiLabelBinarizer(classes=[0,1,2,3,4,5,6,7,8])
    one_hot_labels = mlb.fit_transform(examples['label'])

    model_input['label'] = one_hot_labels
    # model_input['n_label'] = []
    # for one_hot_label in one_hot_labels:
    #     if one_hot_label[-1] == 0:
    #         model_input['n_label'].append(sum(one_hot_label) - 1)
    #     else:
    #         model_input['n_label'].append(0)
    model_input['n_label'] = [sum(one_hot_label[:-1]) for one_hot_label in one_hot_labels] # 이거도 가능할 듯
    # model_input['n_label'] = [sum(one_hot_label) - 1 for one_hot_label in one_hot_labels]

    return model_input

In [10]:
print(train)

tokenized_train = train.map(tokenize_function, batched=True)
tokenized_valid = validation.map(tokenize_function, batched=True)

print(tokenized_train)

Dataset({
    features: ['text', 'label'],
    num_rows: 78977
})
Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'n_label'],
    num_rows: 78977
})


## 모델 학습

In [11]:
from transformers import TrainingArguments

BATCH_SIZE = 64 # 32
EPOCHS = 1

save_dir = '/root/korean_hate_speech_detection/model'

training_args = TrainingArguments(
    output_dir=save_dir,
    do_train=True,
    do_eval=True,
    # save_steps=999999999,
    evaluation_strategy="epoch",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_strategy="epoch",
    logging_dir='./logs',
    learning_rate=2e-5,
    run_name="v1",
    save_strategy="no",
    label_names=['labels', 'n_label'],
    seed=42,
)

In [12]:
import torch
from torchmetrics import Accuracy, F1Score, HammingDistance, AUROC, ExactMatch

def compute_metrics(eval_pred):
    threshold = 0.5

    # model_output = eval_pred.predictions
    # labels = eval_pred.label_ids
    intent_logits, n_logits = eval_pred.predictions
    intent_labels, n_labels = eval_pred.label_ids


    intent_logits = torch.Tensor(intent_logits)
    intent_labels = torch.Tensor(intent_labels).long()

    n_logits = torch.Tensor(n_logits)
    n_labels = torch.Tensor(n_labels).long()

    n_pred = torch.argmax(n_logits, dim=1)

    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(intent_logits))

    preds = torch.zeros(size = probs.size())
    preds[probs >= threshold] = 1

    ...

    for b in range(preds.shape[0]):

        # hate: 1, no_hate: 1
        if preds[b][8] == 1:
            preds[b][:-1] = 0

        # hate: 0, no_hate: 0
        if preds[b][:-1].sum() == 0:
            preds[b][8] = 1

        # hate: 1, no_hate: 1
        if preds[b][:-1].sum() != 0:
            preds[b][8] = 0



    accuracy = Accuracy(task='multilabel', num_labels=9)
    f1_macro = F1Score(task="multilabel", num_labels=9, average='macro')
    f1_micro = F1Score(task="multilabel", num_labels=9, average='micro')
    f1_weight = F1Score(task="multilabel", num_labels=9, average='weighted')
    em = ExactMatch(task='multiclass', num_classes=2)
    auroc = AUROC(task='multilabel', num_labels=9, average='micro')
    hamming = HammingDistance(task="multiclass", num_classes=2)
    n_int_accuracy = Accuracy(task='multiclass', num_classes=9)

    f1s = F1Score(task='multilabel', num_labels=9, average=None)

    print("f1s")
    print(f1s(preds, intent_labels))


    metrics = {'accuracy': accuracy(preds, intent_labels),
               'f1_macro': f1_macro(preds, intent_labels),
               'f1_micro': f1_micro(preds, intent_labels),
               'f1_weighted': f1_weight(preds, intent_labels),
               'auroc': auroc(preds, intent_labels),
               'hamming_loss': hamming(preds, intent_labels),
               'em': em(preds, intent_labels),
               'n_int_accuracy': n_int_accuracy(n_pred, n_labels)}
    return metrics

In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Micro,F1 Weighted,Auroc,Hamming Loss,Em,N Int Accuracy,Runtime,Samples Per Second,Steps Per Second
1,0.735400,0.609073,0.902917,0.150252,0.590034,0.452693,0.754375,0.097083,0.582612,0.799225,16.157400,543.157000,8.541000


f1s
tensor([0.0000, 0.0000, 0.0089, 0.5784, 0.0000, 0.0000, 0.0000, 0.0000, 0.7649])


TrainOutput(global_step=1235, training_loss=0.7354170455623735, metrics={'train_runtime': 399.713, 'train_samples_per_second': 197.584, 'train_steps_per_second': 3.09, 'total_flos': 0.0, 'train_loss': 0.7354170455623735, 'epoch': 1.0})

In [15]:
# trainer.save_model()
import os
torch.save(model.state_dict(), os.path.join(f"{save_dir}", "model.pt"))

## 예측

In [16]:
tokenized_test = test.map(tokenize_function, batched=True)

state_dict = torch.load(f"{save_dir}/model.pt")

model = Model()
model.load_state_dict(state_dict=state_dict)
model.to(device)

Map:   0%|          | 0/21939 [00:00<?, ? examples/s]

Model(
  (model): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [18]:
trainer.evaluate()

f1s
tensor([0.0000, 0.0000, 0.0073, 0.5944, 0.0000, 0.0000, 0.0000, 0.0000, 0.7523])


{'eval_loss': 0.616534411907196,
 'eval_accuracy': 0.8985520601272583,
 'eval_f1_macro': 0.15044835209846497,
 'eval_f1_micro': 0.5735998749732971,
 'eval_f1_weighted': 0.43247219920158386,
 'eval_auroc': 0.7445394396781921,
 'eval_hamming_loss': 0.1014479398727417,
 'eval_em': 0.5630612373352051,
 'eval_n_int_accuracy': 0.7946578860282898,
 'eval_runtime': 40.3297,
 'eval_samples_per_second': 543.991,
 'eval_steps_per_second': 8.505}